# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../Weatherpy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (m/s),Country,Date
0,0,sesheke,-17.4759,24.2968,24.63,50,29,2.42,ZM,1702321373
1,1,edinburgh of the seven seas,-37.0676,-12.3116,13.82,83,100,9.76,SH,1702321375
2,2,malanje,-9.5402,16.3410,17.54,97,100,0.56,AO,1702321377
3,3,adamstown,-25.0660,-130.1015,21.45,61,2,2.39,PN,1702321377
4,4,tiksi,71.6872,128.8694,-22.44,100,100,4.08,RU,1702321377


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(
"Lng",
"Lat",
geo = True,
size = "Humidity (%)",
scale = .8,
color = "City",
tiles = "OSM",
frame_width = 800,
frame_height = 600,
alpha=0.5)

# Display the map
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity (%))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_city_df = city_data_df.loc[(city_data_df['Max Temp (C)']<=27)    #Max temp less than or equal to 27
                            &(city_data_df['Max Temp (C)']>=14)        #Max temp greater than or equal to 14
                            &(city_data_df['Cloudiness (%)']<=50)      #Cloudiness less than or equal to 50
                            &(city_data_df['Humidity (%)']<55)         #Humidity less than 55
                            &(city_data_df['Humidity (%)']>25)         #Humidity more than 25
                            &(city_data_df['Wind Speed (m/s)']<3.5),:]   #Wind speed less than 3.5 m/s


# Drop any rows with null values
ideal_city_df.dropna()
city_count = len(ideal_city_df)

# Display sample data
print (f'Total ideal cities: {city_count}')
ideal_city_df.head()

Total ideal cities: 12


,City_ID,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (m/s),Country,Date
0,0,sesheke,-17.4759,24.2968,24.63,50,29,2.42,ZM,1702321373
66,66,maridi,4.9158,29.4742,26.76,44,19,1.00,SS,1702321390
159,159,galliano,29.4422,-90.2992,15.00,31,0,3.09,US,1702321404
274,274,los osos,35.3111,-120.8324,23.91,29,8,1.79,US,1702321206
434,434,diboll,31.1871,-94.7810,17.16,28,0,0.00,US,1702321436


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity (%),Hotel Name
0,sesheke,ZM,-17.4759,24.2968,50,
66,maridi,SS,4.9158,29.4742,44,
159,galliano,US,29.4422,-90.2992,31,
274,los osos,US,35.3111,-120.8324,29,
434,diboll,US,31.1871,-94.7810,28,
449,riosucio,CO,5.4216,-75.7032,49,
451,illizi,DZ,26.4833,8.4667,35,
469,sironj,IN,24.1000,77.7000,44,
500,penon blanco,MX,24.7897,-104.0369,33,
515,ghat,LY,24.9647,10.1728,34,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',
          'apiKey':geoapify_key,
          'limit':1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sesheke - nearest hotel: Sisheke Lodge
maridi - nearest hotel: No hotel found
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
los osos - nearest hotel: Back Bay Inn
diboll - nearest hotel: No hotel found
riosucio - nearest hotel: Hotel Real Plaza
illizi - nearest hotel: بوناقة
sironj - nearest hotel: No hotel found
penon blanco - nearest hotel: No hotel found
ghat - nearest hotel: فندق تاسيلي
chandbali - nearest hotel: No hotel found
giridih - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity (%),Hotel Name
0,sesheke,ZM,-17.4759,24.2968,50,Sisheke Lodge
66,maridi,SS,4.9158,29.4742,44,No hotel found
159,galliano,US,29.4422,-90.2992,31,Fairfield Inn & Suites by Marriott Cut Off-Gal...
274,los osos,US,35.3111,-120.8324,29,Back Bay Inn
434,diboll,US,31.1871,-94.7810,28,No hotel found
449,riosucio,CO,5.4216,-75.7032,49,Hotel Real Plaza
451,illizi,DZ,26.4833,8.4667,35,بوناقة
469,sironj,IN,24.1000,77.7000,44,No hotel found
500,penon blanco,MX,24.7897,-104.0369,33,No hotel found
515,ghat,LY,24.9647,10.1728,34,فندق تاسيلي


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
#%%capture --no-display

# Configure the map plot
hotels_map = hotel_df.hvplot.points(
"Lng",
"Lat",
geo = True,
size = "Humidity (%)",
scale = 1.5,
color = "City",
hover_cols = ["Hotel Name", "Country"],
frame_width = 800,
frame_height = 600,
tiles = "OSM",
alpha=0.5)

# Display the map
hotels_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity (%),Hotel Name,Country)